# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [2]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string


stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\allen\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [3]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
gb = GradientBoostingClassifier()
param = {'n_estimators': [100,150], 'max_depth': [7,11,15],'learning_rate': [0.1]}
gs = GridSearchCV(gb, param, cv = 5, n_jobs = 1)
cv_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score',ascending = False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,367.978198,18.472695,0.197512,0.007774,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.968582,0.976661,0.970350,0.966757,0.968553,0.970181,0.003434,1
3,245.059748,0.793922,0.165175,0.003494,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964991,0.977558,0.969452,0.969452,0.968553,0.970001,0.004121,2
1,187.853321,7.751631,0.189223,0.011701,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.963196,0.980251,0.970350,0.966757,0.966757,0.969462,0.005850,3
0,119.041095,6.878443,0.172065,0.007060,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.962298,0.978456,0.970350,0.965858,0.968553,0.969103,0.005407,4
4,205.918514,4.679875,0.177029,0.013217,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.965889,0.974865,0.969452,0.967655,0.965858,0.968744,0.003336,5


In [6]:
gb = GradientBoostingClassifier()
param = {'n_estimators': [100,150], 'max_depth': [7,11,15],'learning_rate': [0.1]}
gs = GridSearchCV(gb, param, cv = 5, n_jobs = 1)
cv_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score',ascending = False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,364.681548,2.632435,0.190287,0.002864,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964991,0.977558,0.968553,0.965858,0.970350,0.969462,0.004475,1
3,307.793103,5.567822,0.212383,0.015531,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.962298,0.976661,0.967655,0.965858,0.971249,0.968744,0.004899,2
2,195.755850,0.970741,0.187377,0.003778,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964093,0.977558,0.967655,0.963163,0.971249,0.968744,0.005252,3
0,126.539664,1.090550,0.169980,0.004070,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.978456,0.970350,0.961366,0.968553,0.968564,0.005880,4
1,199.516754,5.197680,0.181909,0.005674,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.978456,0.967655,0.964960,0.967655,0.968564,0.005147,4
